In [3]:
import pickle
import pandas as pd
import re
from nn_model.globals import MODEL_SIZES, NEURON_SELECTION
import numpy as np
from scipy import sparse

Converts connections from mozaik dataset (https://gin.g-node.org/lucabaroni/LSV1M_Dic23_Cortical) to a compatible format

In [4]:
def extract_real_connections(fname='exported_connections.pickle'):
    with open(fname, 'rb') as f:
        d = pickle.load(f)

    d = {k.removeprefix('V1').removesuffix('Connection'): v for k, v in d.items() if 'Aff' not in k}

    translate = {
        "L4Exc": 'V1_Exc_L4',
        "L4Inh": 'V1_Inh_L4',
        "L23Exc": 'V1_Exc_L23',
        "L23Inh": 'V1_Inh_L23',
    }
    pattern = re.compile(r'^(L\d+(?:Exc|Inh))(L\d+(?:Exc|Inh))$')

    res = {}

    for k, v in d.items():
        # population names
        m = pattern.match(k)
        pre, post = m.groups()
        pre = translate[pre]
        post = translate[post]
        # sizes
        pre_size = MODEL_SIZES[pre]
        post_size = MODEL_SIZES[post]
        # neuron subsets
        pre_filter = NEURON_SELECTION[pre].astype(int)
        post_filter = NEURON_SELECTION[post].astype(int)
        # dataframe of connections
        df = pd.DataFrame(v, columns=['pre', 'post', 'w'])
        df.pre = df.pre.astype(int)
        df.post = df.post.astype(int)
        # select the neuron subset
        df = df[df.pre.isin(pre_filter) & df.post.isin(post_filter)]
        # ensure that the indices arent out of range
        # [59, 47, 99, 99] -> [1, 0, 2, 2]
        df['pre'] = df.pre.rank(method='dense').astype(int) - 1
        df['post'] = df.post.rank(method='dense').astype(int) - 1
        # choose subset
        df = df[df.post.isin(post_filter)]
        mat = np.zeros((pre_size, post_size))
        # create dense ndarray
        mat[df.pre, df.post] = df.w
        res[(pre, post)] = mat
    return res

In [5]:
res = extract_real_connections()

In [6]:
res

{('V1_Exc_L4',
  'V1_Exc_L4'): array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], shape=(3750, 3750)),
 ('V1_Exc_L4',
  'V1_Inh_L4'): array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], shape=(3750, 937)),
 ('V1_Inh_L4',
  'V1_Exc_L4'): array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], shape=(937, 3750)),
 ('V1_Inh_L4',
  'V1_Inh_L4'): array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.